# GitHub repo: https://github.com/mehrdad-dev/Persian-character-level-poetry-generation

# Libraries

In [ ]:
import os
import re
import json
import numpy as np
from tqdm import tqdm

import torch
import torch.nn as nn
from torch.nn import functional as F
from torch.utils.data import Dataset
from torch.utils.data import DataLoader
from torch.optim.lr_scheduler import OneCycleLR
from torch.nn.utils.rnn import pad_sequence

# Dataset


kaggle link: https://www.kaggle.com/datasets/akhorshidi/hafez?select=hafez.txt

# Clean our dataset (Hafez.txt)

In [ ]:
input_file_path = 'hafez.txt'
output_file_path = 'preprocessed_hafez.txt'

with open(input_file_path, 'r', encoding='utf-8') as input_file, open(output_file_path, 'w', encoding='utf-8') as output_file:
    for line in input_file:
        if not re.search('[۰-۹]', line):
            output_file.write(line)

os.rename(output_file_path, input_file_path)  # rename if you want

# Read the dataset

In [ ]:
def read_file(filename):
    with open(filename) as f:
        corpus = f.readlines()
        corpus = [sent.strip() for sent in corpus if len(sent) > 5]
    return corpus

corpus =  read_file("hafez.txt")

# Helper Functions

In [ ]:
def char_tensor(string, mapping):
    tensor = torch.zeros(len(string)).long()
    for c in range(len(string)):
        tensor[c] = mapping['stoi'][string[c]]
    return tensor

In [ ]:
def train(inp, target):
    hidden = model.init_hidden(batch_size)
    hidden = (hidden[0].cuda(), hidden[1].cuda())
    model.zero_grad()
    outputs, hidden = model(inp, hidden)  # (B, T, C)
    b, t, c = outputs.shape
    logits = outputs.reshape(-1, c)  # (B*T,C)
    loss = criterion(logits, target.view(-1))
    loss.backward()
    optimizer.step()
    return loss

In [ ]:
def generate(model, mapping, prime_str='', predict_len=100, temperature=0.8):
    itos = mapping['itos']
    itos = {int(key):item for key,item in itos.items()}
    prime_input = char_tensor(prime_str, mapping)
    prime_input = prime_input.unsqueeze(0)
    predicted = prime_str
    hidden = model.init_hidden(1)
    hidden = (hidden[0].cuda(), hidden[1].cuda())
    prime_input = prime_input.cuda()
    for p in range(len(prime_str) - 1):
        _, hidden = model(prime_input[:, p].unsqueeze(1), hidden)
    inp = prime_input[:, -1].unsqueeze(-1)
    for p in range(predict_len):
        output, hidden = model(inp, hidden)
        output_dist = output.data.view(-1).div(temperature).exp()
        top_i = torch.multinomial(output_dist, 1)[0]
        predicted_char = itos[int(top_i)]
        predicted += predicted_char
        inp = char_tensor(predicted_char, mapping).unsqueeze(-1)
        inp = inp.cuda()
    return predicted

# Data loader

In [ ]:
class CharDataset(Dataset):
    def __init__(self, data, sequence_lenght):
        self.data = "\n".join(data)
        self.vocabulary = sorted(list(set(self.data)))
        data_size, self.n_characters = len(self.data), len(self.vocabulary)
        self.itos = {i: ch for i, ch in enumerate(self.vocabulary)}
        self.stoi =  {ch: i for i, ch in enumerate(self.vocabulary)}
        self.mapping = {'stoi': self.stoi, 'itos': self.itos}
        with open('mapping.json', 'w') as outfile:
            json.dump(self.mapping , outfile)
        self.sequence_lenght = sequence_lenght

    def __len__(self):
        return len(self.data) - self.sequence_lenght

    def __getitem__(self, idx):
      chunk = self.data[idx:idx + self.sequence_lenght + 1]
      dix = [self.stoi[s] for s in chunk]
      x = torch.tensor(dix[:-1], dtype=torch.long)
      y = torch.tensor(dix[1:], dtype=torch.long)
      x = x.cuda()
      y = y.cuda()
      return x, y

# Define our model

In [ ]:
class CharRNN(nn.Module):
    def __init__(self, input_size, hidden_size, output_size, dropout=0.5, n_layers=1):
        super(CharRNN, self).__init__()
        self.input_size = input_size
        self.hidden_size = hidden_size
        self.output_size = output_size
        self.n_layers = n_layers

        self.rnn = nn.LSTM(input_size=self.hidden_size, hidden_size=self.hidden_size,
                           num_layers=self.n_layers, batch_first=True)
        self.decoder = nn.Linear(hidden_size, output_size)
        self.dropout = nn.Dropout(p=dropout)

    def forward(self, input, hidden):
        encoded = F.one_hot(input, num_classes=self.hidden_size)
        encoded = encoded.type(torch.float)
        output, hidden = self.rnn(encoded, hidden)
        output = self.dropout(output)
        output = self.decoder(output)
        return output, hidden

    def init_hidden(self, batch_size):
          return (torch.zeros(self.n_layers, batch_size, self.hidden_size),
                  torch.zeros(self.n_layers, batch_size, self.hidden_size))

In [ ]:
batch_size= 128
hidden_size= 1024
n_layers= 3
learning_rate= 0.001
n_epochs= 20
dropout= 0.5
sequence_lenght= 50

In [ ]:
dataset = CharDataset(data=corpus, sequence_lenght=sequence_lenght)
train_dataloader = DataLoader(dataset, batch_size=batch_size, shuffle=False)

model = CharRNN(dataset.n_characters, hidden_size, dataset.n_characters,
                dropout=dropout, n_layers=n_layers)
model.cuda()

optimizer = torch.optim.Adam(model.parameters(), lr=learning_rate)
criterion = nn.CrossEntropyLoss()

# Training loop

In [ ]:
all_train_loss = []
for epoch in tqdm(range(1, n_epochs + 1)):
    batch_number = 0
    train_loss_avg = 0

    for x,y in iter(train_dataloader):
        if len(x) == batch_size:
            train_loss = train(x,y)
            train_loss_avg += train_loss
            batch_number += 1
            prompt = "عجب از لطف تو"
            if batch_number % 200 == 0:
              print(train_loss)
              print(generate(model,dataset.mapping ,f'{prompt}', 200), '\n')
              print("-----------------")

    all_train_loss.append(train_loss_avg/(len(train_dataloader)-1))
    if epoch % 5 == 0:
        torch.save(model, f"hafez-{epoch}.pt")

  0%|          | 0/20 [00:00<?, ?it/s]

tensor(2.5200, device='cuda:0', grad_fn=<NllLossBackward0>)
عجب از لطف تو خو اه گاش واژ کبت خم صا اری ا ام آلا دين رف سن اه بسسی بيش قسال پوون صا شه چر
‌رل اات
هست
مر ار گان وسيهی ه م يت م ت خرگ مر دم کيدی
او  اي نوم جبا  مسف صسل کف ره آسيه هو کم با ه دز
صام موت
رنس
اش تل‌ 

-----------------
tensor(2.5828, device='cuda:0', grad_fn=<NllLossBackward0>)
عجب از لطف تو‌خنرت سمت
نه يز بل هه می‌حبخ آف باش مخدک اات آر ته کااشان وکتوو ده‌ب ز تد
ز
تهد سره کو منار خزات
 ده رو تير منيات بهاد
سستاد
صاس کاام مواار گه مر جزا منمت بفا اين تو منات که ايحياما لن‌ف
حاای آف خب ان 

-----------------
tensor(2.3847, device='cuda:0', grad_fn=<NllLossBackward0>)
عجب از لطف تواو به اه دلممان به ماامم که نين که تافت صو در مو به مطتال که خنآر که ظر بر کرد
ن که نود
که خا چه چر کرن
ار کرر دل و خا تو دل که دل نی ول ده دلدم که شاف اداد
کان خلير که قنت اناان بار بفت
در تاد چو آر  

-----------------
tensor(2.3906, device='cuda:0', grad_fn=<NllLossBackward0>)
عجب از لطف تو دج تواش کی دلد
که دو دير که می‌پاان ز چشط دي

  5%|▌         | 1/20 [10:58<3:28:30, 658.43s/it]

tensor(1.8049, device='cuda:0', grad_fn=<NllLossBackward0>)
عجب از لطف تو زان عيش زور با بس خوش در آن ترسي تو وصل هماش است
از در بر به ياد خواج سر عالا چه خوش که کران و دور و بگرين باد غلام مجز عشق
نقش نه عيش نقه تو به حاجت حافظ نه سخن که خوش و ما ز ماه از نياز
عاق است سحر 

-----------------
tensor(2.2308, device='cuda:0', grad_fn=<NllLossBackward0>)
عجب از لطف تو کرشت
بازم که گذر سنگين اندر می‌فرستی
گفت
بر گرفتشون از آب گرفت
تو دما که نگرفت خدا آن گفت
پاديانه‌ها دست را بوی گفت جاست
که ای دعا که که که به رفت
چند راز خبته که‌ای که من دلی
نيست تو در آن بايد می‌خ 

-----------------
tensor(1.9727, device='cuda:0', grad_fn=<NllLossBackward0>)
عجب از لطف تو بر نتوان عيش خور حافظ
اتنمماين خواهم جان در جهان ای مهر ننداد
من که در اين غمزه سوی هر که لنظر خور که پا راه خواهم کرد
ره بر بر اين شد هزار دل بخت حر راز گرفت
که چاره عجل که اندر لعلی روشن بهشت
که با 

-----------------
tensor(1.9026, device='cuda:0', grad_fn=<NllLossBackward0>)
عجب از لطف تو به ترکان آمد
باز صافی چند پيايت طبه نستد
نسي

 10%|█         | 2/20 [21:57<3:17:40, 658.89s/it]

...

 90%|█████████ | 18/20 [3:18:09<22:01, 660.86s/it]

tensor(0.4307, device='cuda:0', grad_fn=<NllLossBackward0>)
عجب از لطف تو را زين عميق
در همه عالم هيچ درد و دو عالمی باده خورد
اين چه استاده‌ام بر رخ زنده روان خوانديد
گو در که جام زر است
شرح شکن زلف خاصه و ابروی دل مصری فتاده در وقاين جاست
چون نقش غمگيرم به غصه دو جان کار 

-----------------
tensor(0.4072, device='cuda:0', grad_fn=<NllLossBackward0>)
عجب از لطف توان برد
ای دولت سير سرخ است و کرامت
کوته نکند بحث سر زلف تو حافظ
پيوسته شد اين سلسله تا روز قيامت
ای هدهد صبا به سبا می‌فرستمت
تا مطربان ز شوق منت آگهی روم تا به سير و سخن گات
به مهربانی محبت است که در 

-----------------
tensor(0.4465, device='cuda:0', grad_fn=<NllLossBackward0>)
عجب از لطف توام ما ز دلش کين
که ساز من محروم چشم آن ابروکمان کرد
ياد باد آن که ز ما وقت سحر شکايت با آن خاره کرد
خوشا نماز که مردافت
حيش در دلم ز اس خبر و خون بخت سوخت
تورنار دل شاه نشان است
که تير چون آه زمی خناف 

-----------------
tensor(0.4611, device='cuda:0', grad_fn=<NllLossBackward0>)
عجب از لطف تو آبی که شکل بار کمتر می‌کنند
صد ملک سليمانه ز

 95%|█████████▌| 19/20 [3:29:11<11:01, 661.17s/it]

tensor(0.4308, device='cuda:0', grad_fn=<NllLossBackward0>)
عجب از لطف تو روز خويشتن است
به معد و فتوی در کمر سرو بلندش باد جان نالا براندريايی
رقيب اغلان می‌روند اين هظام انداخت
دلبر ما فتاد در اين کس که هيچ نگار
ای بسا تو که قصه فاش از غم هر آن کس که از او بوسی
تا دهان م 

-----------------
tensor(0.4493, device='cuda:0', grad_fn=<NllLossBackward0>)
عجب از لطف توان سوخت همچو عود
می ده که عاشقان نظرباز می‌اش در مکن است در باد ما رفت
دور دارد اين حکايت
ای آفتاب خوبان می‌دهد کن
که در دست زلف و ريا نمايد بستان تو
بر بوی آن که جرعه جامت به ما رسد
بر بوی آن که جشوه 

-----------------
tensor(0.3894, device='cuda:0', grad_fn=<NllLossBackward0>)
عجب از لطف توام نشين داری
که تريب ققت عروق حافظ و فرياد که
اگر نه دست در حلقه آن زلف دوتا نتوان کرد
من چه گويم که می‌کرد اين جا ببرد
احميت عفض کن بر مجلس عشق ناز برد
ور ز نو آب که بود عارفا نکرد
من از اين بمار مکن 

-----------------
tensor(0.4800, device='cuda:0', grad_fn=<NllLossBackward0>)
عجب از لطف تو آن خسرو شيرين آمد
رود به قد و چهره عاشقانه ب

100%|██████████| 20/20 [3:40:12<00:00, 660.65s/it]


# Generate

In [ ]:
# load model if you need
PATH = "/content/hafex.pt"
model = torch.load(PATH)

In [ ]:
mapping = open('/content/mapping.json')
mapping = json.load(mapping)
generate(model, mapping, prime_str="عشق")

'عشق بازآيی\nدايره جفا کرد تو بنده زار و می\nشوق خرده مگير و من به طغار دوست دور\nکاين گويی نمی\u200cزنی\nباز ظفر'